In [ ]:
# import all the required packages
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from collections import Counter
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# Open whatsapp chat which is named as _chat by default
with open('_chat.txt') as f:
    lines = [line.rstrip('\n') for line in f]

In [ ]:
# we are slippting every line using ']' as the splitter
# this create a list which has date time stamp and message of the persons alternatively
new_split_lines = []
for line in lines:
    line = line.strip()
    new_split_lines.extend(re.split(']',line))  

In [ ]:
# this will just strip any trailling and leading spaces for all the members in the list
for i in np.arange(0, len(new_split_lines)):
    new_split_lines[i] = new_split_lines[i].strip()  

In [ ]:
# this is used to know about number of messages send by each user and number of timestamps
time_list = []
persons_list = []
person1_list = []
person2_list = []

person1 = '<person1_name in the chat>'
person2 = '<person2_name in the chat>'

count_empty_lines = 0.0
old_split_lines = new_split_lines.copy()
p1 = 0
p2 = 0

for line in old_split_lines:
    if len(line) == 0:
        count_empty_lines += 1 #610
        continue
    if line[0] == '[':
        if(line[-3:] in (' AM', ' PM')):
            time_list.append(line[-((len(line))-1):])
    else:
        line = line.strip()
        if(str(line[:len(person1)]) == person1):
            p1 = 1
            p2 = 0
            person1_list.append(line)
        elif(str(line[:len(person2)]) == person2):
            p1 = 0
            p2 = 1
            person2_list.append(line)
        else:
            if p1:
                person1_list.append(line) 
            if p2:
                person2_list.append(line)
print(person1 ," messages: ", len(person1_list), sep="")
print(person2, " messages: ", len(person2_list)) 
print("Number of time stamps: ", len(time_list)) 

In [ ]:
# this block of code is used to create same length for person 1 messages and person 2 messages
# w.r.t. number of time stamps by introducing empty strings in the lists when the person has not message
# for that time stamp.
time_list = []
persons_list = []
person1_list = []
person2_list = []
old_split_lines = []
old_split_lines1 = new_split_lines.copy()
p1 = 0
p2 = 0

for index in range(0, len(old_split_lines1)):
    line = old_split_lines1[index]
    if len(line) == 0:
        continue
    old_split_lines.append(line)
    
index = 0
while(index != len(old_split_lines)):
    line = old_split_lines[index]
    if line[0] == '[':
        if(line[-3:] in (' AM', ' PM')):
            time_list.append(line[-((len(line))-1):])
        index += 1
        line = old_split_lines[index]
        p1_line = ""
        p2_line = ""
        while((line[-3:] not in (' AM', ' PM'))):
            line = old_split_lines[index]
            line = line.strip()
            if(str(line[:len(person1)]) == person1):
                p1 = 1
                p2 = 0
                p1_line += line + " "
                index += 1
            elif(str(line[:len(person2)]) == person2):
                p1 = 0
                p2 = 1
                p2_line += line + " "
                index += 1
            else:
                if p1:
                    p1_line += line + " "
                    index += 1
                if p2:
                    p2_line += line + " "
                    index += 1
            if(index == len(old_split_lines)):
                break
            line = old_split_lines[index]
        if(len(p1_line) == 0):
            person1_list.append("NNN/AAA")
            p2_line = p2_line[len(person2)+2:len(p2_line)]
            person2_list.append(p2_line)
        if(len(p2_line) == 0):
            p1_line = p1_line[len(person1)+2:len(p1_line)]
            person1_list.append(p1_line)
            person2_list.append("NNN/AAA")
        if(index == len(old_split_lines)):
                break
        if(old_split_lines[index][0]!="["):
            index += 1
            continue
    else:
     index += 1 

print(len(time_list))
print(len(person1_list))
print(len(person2_list))

In [ ]:
se1 = pd.Series(time_list)
se2 = pd.Series(person1_list)
se3 = pd.Series(person2_list)
df = pd.DataFrame()
df['time'] = se1.values
df['person1'] = se2.values
df['person2'] = se3.values
df = df.replace('NNN/AAA',np.NaN)

In [ ]:
# converting time column which was in string to a pandas datatime column
df['time'] = pd.to_datetime(df['time'])
# converting datetime to timeindex in IST
df_timeindex = pd.DatetimeIndex(df['time']).tz_localize('UTC').tz_convert('Asia/Kolkata')

In [ ]:
df_person1 = pd.DataFrame({'date_time': df_timeindex})
df_person1['date_time'] = pd.to_datetime(df_person1['date_time'])
df_person1['person'] = df['person1'].copy()
df_person1 = df_person1.dropna(axis=0, how='any')
df_person1.index = df_person1['date_time']
del(df_person1['date_time'])
df_person1['message'] = 1

In [ ]:
df_person2 = pd.DataFrame({'date_time': df_timeindex})
df_person2['date_time'] = pd.to_datetime(df_person2['date_time'])
df_person2['person'] = df['person2'].copy()
df_person2 = df_person2.dropna(axis=0, how='any')
df_person2.index = df_person2['date_time']
del(df_person2['date_time'])
df_person2['message'] = 1

In [ ]:
number_of_words = [] # number of words in a message
words = [] # stores message words in a list
for row in df_person1['person']:
    row = row.rstrip()
    data = re.split(' ',row)
    number_of_words.append(len(data))
    words.append(data)
    
df_person1['words'] = words
df_person1['number_of_words'] = number_of_words

number_of_words = [] # number of words in a message
words = [] # stores message words in a list
for row in df_person2['person']:
    row = row.rstrip()
    data = re.split(' ',row)
    number_of_words.append(len(data))
    words.append(data)
    
df_person2['words'] = words
df_person2['number_of_words'] = number_of_words

In [ ]:
#Analysis of Data

print("Duration of Data: ")
print("Start Date: ", df['time'][0])
print("End Date: ", df['time'][len(df)-1])

total_messages_person1 = np.sum(df_person1['message'])
total_messages_person2 = np.sum(df_person2['message'])
total_messages = total_messages_person1 + total_messages_person2
print("Total number of messages: ", total_messages)
print("Total number of messages sent by ", person1, ": ", total_messages_person1)
print("Percentage of messages sent by ", person1, ": ", (total_messages_person1/total_messages)*100)
print("Total number of messages sent by ", person2, ": ", total_messages_person2)
print("Percentage of messages sent by ", person2, ": ", (total_messages_person2/total_messages)*100)

total_words_person1 = np.sum(df_person1['number_of_words'])
total_words_person2 = np.sum(df_person2['number_of_words'])
total_words = total_words_person1 + total_words_person2
print("Total number of words in all messages: ", total_words)
print("Total number of words sent by ", person1, ": ", total_words_person1)
print("Percentage of words sent by ", person1, ": ", (total_words_person1/total_words)*100)
print("Total number of words sent by ", person2, ": ", total_words_person2)
print("Percentage of words sent by ", person2, ": ", (total_words_person2/total_words)*100)
print("Most number of messages by ", person1, " in 1 day: ", df_person1.resample('d').sum().message.max())
print("Most number of messages by ", person2, "in 1 day: ", df_person2.resample('d').sum().message.max())
print("Most number of messages by ", person1, "in 1 month: ", df_person1.resample('m').sum().message.max())
print("Most number of messages by ", person2, "in 1 month: ", df_person2.resample('m').sum().message.max())
print("Most number of messages by ", person1, "in 1 year: ", df_person1.resample('a').sum().message.max())
print("Most number of messages by ", person2, "in year: ", df_person2.resample('a').sum().message.max())
# len(df_timestamp1['2015'])
# len(df_timestamp1['2016-10'])
# len(df_timestamp1['2016-10-05'])
# len(df_timestamp1['5/3/2016':'5/4/2016'])

list_words_person1 = []
for row in df_person1['words']:
    list_words_person1.extend(row)

list_words_person2 = []
for row in df_person2['words']:
    list_words_person2.extend(row)

counts_person1 = Counter(list_words_person1)
print("Most common words by ", person1, ": ", counts_person1.most_common(10))
counts_person2 = Counter(list_words_person2)
print("Most common words by ", person2, ": ", counts_person2.most_common(10))

In [ ]:
plt.plot(df_person2.resample('m').sum().message, 'r')
plt.plot(df_person1.resample('m').sum().message, 'b')
plt.xlabel("month")
plt.ylabel("messages per month")
plt.show()
plt.plot(df_person2.resample('w').sum().message, 'r')
plt.plot(df_person1.resample('w').sum().message, 'b')
plt.xlabel("week")
plt.ylabel("messages per week")
plt.show()
plt.plot(df_person2.resample('d').sum().message, 'r')
plt.plot(df_person1.resample('d').sum().message, 'b')
plt.xlabel("day")
plt.ylabel("messages per day")
plt.show()
plt.plot(df_person2.resample('a').sum().message, 'r')
plt.plot(df_person1.resample('a').sum().message, 'b')
plt.xlabel("Year")
plt.ylabel("messages per year")
plt.show()

In [ ]:
plt.plot(df_person2.resample('m').sum().number_of_words, 'r')
plt.plot(df_person1.resample('m').sum().number_of_words, 'b')
plt.xlabel("month")
plt.ylabel("words per month")
plt.show()
plt.plot(df_person2.resample('w').sum().number_of_words, 'r')
plt.plot(df_person1.resample('w').sum().number_of_words, 'b')
plt.xlabel("week")
plt.ylabel("words per week")
plt.show()
plt.plot(df_person2.resample('d').sum().number_of_words, 'r')
plt.plot(df_person1.resample('d').sum().number_of_words, 'b')
plt.xlabel("day")
plt.ylabel("words per day")
plt.show()
plt.plot(df_person2.resample('a').sum().number_of_words, 'r')
plt.plot(df_person1.resample('a').sum().number_of_words, 'b')
plt.xlabel("Year")
plt.ylabel("words per year")
plt.show()

In [ ]:
# create a string which joins all the messages of a single person's message
complete_messages_person1 = ' '.join(list_words_person1)
complete_messages_person2 = ' '.join(list_words_person2)

In [ ]:
# Creation of wordcloud

__file__ = 'Downloads'
d = path.dirname(__file__)

text = complete_messages_person2

# read the mask image
mask_image = np.array(Image.open(path.join(d, "family_with_kid_mask.png")))

stopwords = set(STOPWORDS)
wc = WordCloud(background_color="white", max_words=2000, mask=mask_image, stopwords=stopwords)
wc.generate(text)

# store to file
wc.to_file(path.join(d, "new_image.png"))
image = wc.to_image()
image.show()

In [ ]:
##############     for finding word in chat
j = 0
k = 0
for line in lines:
    j += 1
    if('lava' in line):
        k += 1
        print(j)
        break #remove break if you want to know all occourences
#     for i in line:
#         if(i == 'cool'):
#             print(j)